In [81]:
import numpy as np
import imageio.v2 as imageio

In [82]:
# Read the .bmp input and output files as a numpy 2D matrix
X = imageio.imread('x1.bmp')
# Convert the image to a NumPy array
X = np.array(X)

Y = imageio.imread('y4.bmp')
Y = np.array(Y)

print('X shape:', X.shape)
print('Y shape:', Y.shape)

X shape: (140, 188)
Y shape: (256, 188)


In [83]:
def isPsevdoInversed(A, A_psevdo_inverse) -> bool:
    #  A * A+ * A = A;
    if not np.allclose(A @ A_psevdo_inverse @ A, A):
        print('A * A+ * A != A')
        return False
    #  A+ * A * A+ = A+
    elif not np.allclose(A_psevdo_inverse @ A @ A_psevdo_inverse, A_psevdo_inverse):
        print('A+ * A * A+ != A+')
        return False
    #  A * A+ - symmetric matrix m x m
    elif not np.allclose(A @ A_psevdo_inverse, (A @ A_psevdo_inverse).T):
        print('A * A+ - not symmetric matrix m x m')
        return False
    #  A+ * A - symmetric matrix n x n
    elif not np.allclose(A_psevdo_inverse @ A, (A_psevdo_inverse @ A).T):
        print('A+ * A - not symmetric matrix n x n')
        return False
    
    return True

In [84]:
def PseudoInverseMatrix_MoorePenrose(A, eps=1e-6, delta=5):
    # Calculate the Moore-Penrose pseudo-inverse of a matrix
    # A: input matrix
    # returns: pseudo-inverse of A
    # eps: tolerance for zero singular values
    print('delta:', delta)

    A0 = A.T @ np.linalg.inv(A @ A.T + delta**2 * np.eye(A.shape[0]))
    print('A0:', A0)
    
    delta = delta / 2
    print('delta:', delta)

    while True:    
        A1 = A.T @ np.linalg.inv(A @ A.T + delta**2 * np.eye(A.shape[0]))
        print('A1:', A1)
        if np.linalg.norm(A0 - A1, ord=2) < eps:
            return A1
        
        delta = delta / 2
        print('delta:', delta)

        A0 = A1

# Test
Atest = np.array([[1, 2, 3], [4, 5, 6]])
Atest_pinv = PseudoInverseMatrix_MoorePenrose(Atest)
print('Atest:', Atest)
print('Atest_pinv:', Atest_pinv)
assert isPsevdoInversed(Atest, Atest_pinv)

delta: 5
A0: [[-0.00880162  0.04197698]
 [ 0.01489506  0.04434665]
 [ 0.03859174  0.04671632]]
delta: 2.5
A1: [[-0.06761734  0.0740391 ]
 [ 0.00982151  0.05628482]
 [ 0.08726036  0.03853055]]
delta: 1.25
A1: [[-0.24889378  0.15229405]
 [-0.01447423  0.06953922]
 [ 0.21994533 -0.0132156 ]]
delta: 0.625
A1: [[-0.56421046  0.28497941]
 [-0.05818039  0.08866413]
 [ 0.44784967 -0.10765115]]
delta: 0.3125
A1: [[-0.80930643  0.38779137]
 [-0.09228987  0.10315847]
 [ 0.62472668 -0.18147444]]
delta: 0.15625
A1: [[-0.90667952  0.42861403]
 [-0.10585078  0.10889048]
 [ 0.69497795 -0.21083308]]
delta: 0.078125
A1: [[-0.93471669  0.44036684]
 [-0.10975608  0.11053922]
 [ 0.71520454 -0.2192884 ]]
delta: 0.0390625
A1: [[-0.94199391  0.44341726]
 [-0.11076976  0.11096705]
 [ 0.72045439 -0.22148315]]
delta: 0.01953125
A1: [[-0.94383064  0.44418716]
 [-0.11102561  0.11107503]
 [ 0.72177942 -0.2220371 ]]
delta: 0.009765625
A1: [[-0.94429092  0.44438009]
 [-0.11108973  0.11110209]
 [ 0.72211147 -0.2221759

In [85]:
def PseudoInverseMatrix_Greville(A):
    # Calculate the Greville pseudo-inverse of a matrix
    # A: input matrix
    # returns: pseudo-inverse of A
    # eps: tolerance for zero singular values
    m, n = A.shape
    A_pinv = np.zeros((n, m))

    for i in range(m):
        a = A[i, :].reshape(-1, 1)
        if i == 0:
            if np.all(a == 0):
                A_pinv[i, :] = 0
            else:
                A_pinv[i, :] = a.T / np.dot(a.T, a)
        else:
            Z = np.eye(i) - np.dot(A_pinv[:i, :], A[:i, :])
            if np.dot(a.T, Z) @ a > 0:
                A_pinv[i, :] = np.dot(Z, a) / np.dot(a.T, Z @ a)
            else:
                R = np.dot(A_pinv[:i, :], (A_pinv[:i, :]).T)
                A_pinv[i, :] = np.dot(R, a) / (1 + np.dot(a.T, R @ a))

    return A_pinv

# Test
Atest = np.array([[1, 2, 3], [4, 5, 6]])
Atest_pinv = PseudoInverseMatrix_Greville(Atest)
print('Atest:', Atest)
print('Atest_pinv:', Atest_pinv)
assert isPsevdoInversed(Atest, Atest_pinv)

ValueError: could not broadcast input array from shape (3,) into shape (2,)